In [89]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import time
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from difflib import SequenceMatcher
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import re
from difflib import SequenceMatcher
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains import LLMChain
os.environ["OPENAI_API_KEY"] = "sk-"
embeddings = OpenAIEmbeddings()
from langchain.docstore.document import Document



### Helper Functions

In [177]:
with open(file_path, "w") as progress_file:
    progress_file.write(phase)
def fix_collection_name(collection_name):
    # Replace spaces with underscores
    collection_name = collection_name.replace(' ', '_')

    # Add underscore placeholders if the name is too short
    if len(collection_name) < 3:
        collection_name += '_' * (3 - len(collection_name))

    # Cut off the name if it is too long
    if len(collection_name) > 63:
        collection_name = collection_name[:63]

    # Replace consecutive periods with a single period
    collection_name = re.sub(r'\.\.', '.', collection_name)

    # Replace any invalid characters with underscores
    collection_name = re.sub(r'[^a-zA-Z0-9_.-]', '_', collection_name)

    # Ensure that the name starts and ends with an alphanumeric character
    if not collection_name[0].isalnum():
        collection_name = 'X' + collection_name[1:]
    if not collection_name[-1].isalnum():
        collection_name = collection_name[:-1] + 'X'

    return collection_name

with open("indexholders.txt", "a", encoding="utf-8") as f:
    pass

def read_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)


def folder_exists(path):
    return os.path.isdir(path)

def create_progress_folder(user_id,phase):
    folder_path = os.path.join("progress", user_id)
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, "progress.txt")
    with open(file_path, "w") as progress_file:
        progress_file.write(phase)

    
def check_similarity(string1, string2):
    # Convert strings to lowercase to ignore casing
    string1 = string1.lower()
    string2 = string2.lower()

    # Calculate similarity ratio
    similarity_ratio = SequenceMatcher(None, string1, string2).ratio()

    # Check if similarity probability is greater than or equal to 70%
    if similarity_ratio >= 0.7:
        return True
    else:
        return False


def remove_first_line(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # Exclude the first line
    updated_lines = lines[1:]

    with open(file_path, "w") as file:
        file.writelines(updated_lines)

        
        from difflib import SequenceMatcher


def has_similar_word(target_word, word_list):
    threshold = 0.7
    
    for word in word_list:
        similarity = SequenceMatcher(None, target_word, word).ratio()
        if similarity >= threshold:
            return True
    
    return False




### Indentification Bot

In [165]:
def parse_final_answer(conversation):
    for line in conversation.splitlines():
        if line.startswith("Final Answer:"):
            return line.split("Final Answer:")[1].strip()
    return conversation



In [174]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
def identification_bot(query,userid):
    current_timestamp = time.time()
    formatted_timestamp = time.strftime("%Y-%m-%d", time.localtime(current_timestamp))
    vectorstore_folder  = f'db/{userid}'
            
    problems = ' '
    intervention = ' ' 
    conversation=' '
    goals=' ' 
    similar = ' '
    introduction = ' '
    
    
    users = read_file('indexholders.txt')
    if userid in users:
        assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
        interventiondb = Chroma(collection_name=f'x{userid}intervention',persist_directory=f'{vectorstore_folder}/intervention', embedding_function=embeddings)
        vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
        goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)
        
        introduction = ' '.join(vectordb.get()['documents'][-10:])
        problems = ' '.join(assessmentdb.get()['documents'][-15:])
        intervention = ' '.join(interventiondb.get()['documents'][-15:])
        goals = ' '.join(goalsdb.get()['documents'][:])
        similar = ' '.join([doc.page_content for doc in vectordb.similarity_search(query)])


    
    prompt_template = """
    This is the current Date and you dont know the time:  {time}
    You are a CBT Cognitive Behavioral Therapy self-help chatbot. Your name is Diya. You use CBT therapy to users who are suffering from stress and anxiety. CBT stands for cognitive behavioral therapy. It is a type of talking therapy that can help you manage your problems by changing the way you think and behave. It’s based on the idea that your thoughts, feelings, physical sensations, and actions are interconnected and that negative thoughts and feelings can trap you in a negative cycle. CBT aims to help you deal with overwhelming problems in a more positive way by breaking them down into smaller parts. You’ll learn how to change these negative patterns to improve the way you feel. Unlike some other talking treatments, CBT deals with your current problems, rather than focusing on issues from your past. It looks for practical ways to improve your state of mind on a daily basis.
    YOUR MAIN TASK IS TO USE THE PREVIOUS CONVERSATIONS AND THE CURRENT USER QUESTION TO DECIDE WHAT TOOL SHOULD YOU USE OR WHICH BOT TO REFER TO INSTEAD.
    
    This is the information about your pesonality:
    Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
    You should always use the following rules while having the conversations:
    - your questions and conversations should not be very long and be around 50 words.
    - you should always ask one question at a time.
    - you are encouraged to listen so that the users can confide in you.
    
    
    
    YOU HAVE TO FOLLOW THE CONVERSATION IN A CERTAIN ORDER IN THE GIVEN MANNER:
    (STRICTLY FOLLOW THIS FORMAT)
    USER:[this is the user question]
    DIYA: [OBSERVE THE USER QUESTION AND TAKE INTO ACCOUNT THE CONVERSATION HISTORY YOU THINK IS APPROPRIATE]
    ACTION: [Decide which part of the history is relevent and if a part of history is incomplete]
    OBSERVATION: [THIS PART OF HISTORY HAS THE DATA FOR MY ANSWER I CAN ANSWER DIRECTLY]
    (NOTE IF THE HISTORY OF A CERTAIN STAGE IS INCOMPLETE AND IT IS NEEDED OF THE USER QUERY THEN RETURN THE STAGE NAME INSTEAD ELSE RETURN FINAL ANSWER)
    FINAL ANSWER FROM DIYA:
    
    
    EXAMPLE (THESE ARE EXAMPLES TO HELP YOU UNDERSTAND HOW TO USE THE PATTERN)
    ==========================================
    User: Hi what should we do
    Diya: [this seems like introduction observe it,i have come to a conclusion that this is using the introduction phase data]
    observation: [should  I directly answer it or we already have had the conversation before I can recall it from intrdocution phase data and use that in my final answers]
    action: [combine my answer with any relevent data like name from the introduction phase]
    Final Anwer: [your final answer .....]
    
    User: I am [user's name]
    Diya: [User seems to be intrdoucing himself]
    observation: [I should Directly reply to him]
    action: [give Diya Introduction]
    Final Anwer: [your final answer .....]
    
    
    User:[ I am feeling sad] for [_]days
    Diya: [this seems like assessment phase as it deals with identifying problems observe it,i have come to a conclusion that this is using the assessment phase data]
    observation: [I am not an expert so I suggest using the Assessment phase]
    action: [return 'ASSESSMENT' for any sort of discussion that may be related to getting to know the problem]
    Final Anwer: [Assessment]
    
    User: I want to learn to fly
    Diya: [This seems like the user is wanting to tell me his goals]
    observation: [If the user is telling me his goals I think I should save them]
    action: [return phase name 'Goal Setting']
    Final Anwer: [Goal Setting]
    
    User: so you know my issue what to do now
    Diya: [This seems like the user is wanting telling me problem assessment has completed and I understand the user problems already.]
    observation: [I should check the user assessment phase to know the user issues and suggest moving to intervention phase]
    action: [return phase name 'Intervention']
    Final Anwer: [Intervention]
    
    User: [I have been sad] or [I am not well] or [I am depressed] [anything related to discussing a problem]
    Diya: [This seems like the user is wanting to discuss his issue with me]
    observation: [I should check if I have content that can help inside Assessment phase else return the phase name 'Assessment']
    action: [return phase name 'Assessment' if you have no information about handeling the problem]
    Final Anwer: [Assessment]
    
    User:[lets start] or [yes]
    observation:[user wants to initiate a phase of CBT therapy analyze the information I have at my disposal of every stage and come to a conclusion which stage is the user saying to start]
    action:[return phase name the one the user might need]
    Final Answer: [Phase Name]
    ==========================================
    
    (REMEMBER:
    YOU ARE BAD AT ANAYLZING USER'S PROBLEMS YOU NEED THE ASSESSMENT PHASE BOT
    YOU ARE BAD AT INTERVENTION YOU NEED INTERVENTION PHASE BOT
    YOU ARE BAD AT SAVING AND RETURNING GOALS YOU NEED 'GOALS SETTING' OR 'GOALS RETRIVAL' PHASE BOTS)
    
    (KEEP TO THIS FORMAT FOLLOW THE FORMAT STRICTLY)
    THESE ARE THE RELEVENT PARTS OF HISTORY USE THEM ACCORDING TO THE USER QUESTION:
    
    INTRODUCTION: {context}
    
 
    SIMILAR PARTS TO CONVERSATION: {similar}
    
    (FOLLOW THE FORMAT)
    
    USER:{question}
    Observation: (ANALYZE THE QUESTION DO YOU NEED TO GIVE THE NAME OF THE PHASE OR YOU CAN HANDLE IT)
    Action: (DECIDE WHETER YOU ARE CAPABLE OF ANSWERING THE QUESTION YOURSELF OR YOU NEED TO RETURN THE PHASE NAME ONLY)
    (GIVE THE CORRECT FINAL ANSWER THAT SHOULD BE)Final Answer:
    """
    
    indentification = PromptTemplate(
    input_variables=["time","context","question","similar"],
    template=prompt_template
    )
    indentification_prompt = SystemMessagePromptTemplate(prompt=indentification)
    chat_prompt = ChatPromptTemplate.from_messages([indentification_prompt])
    llm = ChatOpenAI(temperature=0)
    message = chat_prompt.format_prompt(time=formatted_timestamp, question=query,similar=similar,context=introduction).to_messages()
    response = llm(message)
    response = parse_final_answer(response.content)
    return response


In [175]:
response = identification_bot("Hi my name is Hamas","A291")
print(response)

DIYA: Hi Hamas, how can I assist you today?


### Introduction Bot

In [163]:


def introduction_bot(query,vectordb,formatted_timestamp,userid):
    name = ''
    users = read_file('indexholders.txt')
    if userid in users:
        name = ' '.join([doc.page_content for doc in vectordb.similarity_search(query,k=7)])
    name += "....."
    INTRODUCTION_PROMPT = """This is the CURRENT DATE AND TIME TODAY: """+formatted_timestamp+""" 
    THIS INFORMATION WILL SHOW THE NAME OF THE USER: """+name+"""
    Have a conversation with a human, answering the following questions as best you can.
    Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.

    You should always use the following rules while having the conversations:
    - your questions and conversations should not be very long and be around 50 words.
    - you should always ask one question at a time.
    - you are encouraged to listen so that the users can confide in you.

    EXAMPLE:
    ===========================================
    You will introduce the following way:
    User:[ user says Hi] or [user greets you]
    Diya: [Greet the user and Introduce yourself and Give an introduction about the CBT phases you will go through and the methods you will use, explain it and ask the user if they want to continue]
    (also ask some information about the user aswell)

    ===========================================

    You can use the following sentences where appropriate:
    Thank you for your answers. It’s nice to meet you. I’m looking forward to working with you.
    Now that I know a little bit about you, let me tell you more about how this chatbot works and what you can expect from it.
    This chatbot will guide you through a series of sessions that will help you learn and practice CBT techniques and exercises to deal with your stress and anxiety.
    Each session will focus on one or more of your goals or problems that you want to work on with the chatbot.
    Each session will consist of some questions, explanations, examples, scenarios, and feedback from the chatbot. You can type your responses in natural language and the chatbot will try to understand and respond to you.
    Each session will also include some homework or tasks for you to do between sessions to practice what you learned and apply it to your real-life situations.
    You can use this chatbot as often as you like, but it is recommended that you do at least one session per week for optimal results.
    You can also ask the chatbot for help, feedback, or suggestions at any time by typing “help”, “feedback”, or “suggestions”.
    You can also stop or pause the chatbot at any time by typing “stop” or “pause”. You can resume the chatbot by typing “resume” or “continue”.
    Do you have any questions about how this chatbot works or what you can expect from it?
    Please type “yes” or “no”.
    ===========================================

    IF YOU THINK THE INTRODUCTIONS HAVE ENDED AND QUESTIONS HAVE ALSO ENDED AND THINK THE USER SHOULD MOVE TO ASSESSMENT PHASE TO HAVE A BETTER UNDERSTANDING TO THE USER PROBLEM ASK THEM SHOULD YOU INITIATE THE ASSESSMENT PHASE.
    ASK THE USER IF HE WISHES TO START THE ASSESSMENT PHASE


    ===================================================
    Relevant pieces of previous conversation:
    {history}
    ===================================================

    user: {input}
    Diya:
    """
    
    if userid not in users:
        texts = text_splitter.create_documents(['HISTORY'])
        Chroma.add_texts(vectordb,texts=[texts[0].page_content])
        vectordb.persist()

    retriever = vectordb.as_retriever(search_kwargs=dict(k=10))
    memory = VectorStoreRetrieverMemory(retriever=retriever)

    llm = ChatOpenAI(temperature=0.6)
    PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=INTRODUCTION_PROMPT
    )
    diyabot = ConversationChain(
        llm=llm, 
        prompt=PROMPT,
        memory=memory,
        verbose=False
    )
    result = diyabot.predict(input=query) 
    Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{result}\n\n"])
    return result

### ASSESSMENT BOT

In [172]:
ASSESSMENT_PROMPT = """Have a conversation with a human, answering the following questions as best you can.
Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.

YOU ARE AN EXPERT IN ANALYZING USER'S PROBLEMS AND SUMMARIZING THEM YOU WILL ASK A SERIES OF QUESTIONS FROM THE USER AND SUMMARIZE THE USER'S PROBLEM AT THE END.
WHEN YOU UNDERSTAND AND SUMMARIZE EVERYTHING YOU WILL SUGGEST TO THE USER THAT LETS MOVE TO THE INTERVENTION PHASE ask them to type it out like 'yes move to intervention phase'
YOU CAN ALSO PROMPT THE USER ABOUT SETTING SOME GOALS

Using the data below try to understand the user problem
====================================================================
This is the relevent pieces of information in history about the user:
{text}
====================================================================

THESE ARE SOME OF THE QUESTIONS AT YOUR DISPOSAL:
What are your main problems or concerns that you want to work on with me?
How long have you been experiencing these problems or concerns?
How often do these problems or concerns occur?
How severe are these problems or concerns on a scale from 0 (not at all) to 10 (extremely)?
How do these problems or concerns affect your daily life, such as your work, relationships, hobbies, etc.?
What are some situations or triggers that make these problems or concerns worse or better?
What are some coping strategies or resources that you use to deal with these problems or concerns?
What are your expectations or hopes from using this chatbot?

Understand the user problem when you have understood it. you can say this: Thank you for your answers. I appreciate your honesty and openness.
You can tell what are your thoughts about the user's situation, based on the relevent pieces of information in history.
give a suggestion to the user.
When you think Assessment about the user problem is complete and you fully understand the user problem. Tell the user about setting some goals and ask him should I initaiate this stage answer with 'yes' or 'no'

This is the current conversation:
User:{input}
Diya:
"""


def Assessment_bot(query,vectordb,formatted_timestamp,userid,trackingdb):
    filepath = f'progress/{userid}/progress.txt'
    with open(file_path, "a") as progress_file:
        progress_file.write("ASSESSMENT")
    vectorstore_folder = f'db/{userid}'
    assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
    docs = [Document(page_content=t) for t in assessmentdb.get()['documents'][:]]
    

    retriever = vectordb.as_retriever(search_kwargs=dict(k=10))
    memory = VectorStoreRetrieverMemory(retriever=retriever)

    PROMPT = PromptTemplate(template=ASSESSMENT_PROMPT, input_variables=["text","input"])
    chain = load_summarize_chain(OpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
    result = chain({"input_documents": docs,"input":query}, return_only_outputs=True)
    Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{result['output_text']}\n\n"])
    return result['output_text'], True    


### GOAL SETTER

In [178]:
def goal_setter(query,userid):
    vectorstore_folder = f'db/{userid}'
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)
            
    purpose = """
        Your task is to identify the goals from the given text and arrange them in a proper format as this:
        goal1: something
        goal2: something
        goal3: something

        return back to the customer in that format only.

        THIS IS AN EXAMPLE:
        =====================================================
        user:I want to improve my self-esteem, reduce anxiety in social situations, challenge negative thoughts, increase motivation for personal interests.
        goal1: Improve my self-esteem.
        goal2: Reduce my anxiety in social situations.\n
        goal3: Challenge and modify negative thoughts about my abilities.\n
        goal4: Increase my motivation to pursue personal interests.\n
        =====================================================


        Here is the User Question: {query}
        Your Answer:
        """
    prompt = PromptTemplate(
    input_variables=["query"],
    template=purpose,
    )
    ask_prompt = SystemMessagePromptTemplate(prompt=prompt)
    chat_prompt = ChatPromptTemplate.from_messages([ask_prompt])
    llm = ChatOpenAI(temperature=0.6)
    result = llm(chat_prompt.format_prompt(query=query).to_messages())
    Chroma.add_texts(goalsdb,texts=[f"USER GOALS: {result.content}\n\n"])
    
    goalsdb.persist()
    goalsdb = None
    return result.content + '\nNow that we have completed the goal setting, let’s move on to the intervention. This is where I will teach you some CBT techniques and exercises that will help you work on your goals Are you ready to start the intervention?Please type “yes” or “no”.'

In [146]:
def goals_retriver(userid):
    vectorstore_folder = f'db/{userid}'
    llm = ChatOpenAI(temperature=0.6)
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)

    purpose = """
        Your task is to tell the user about their goal. 
        This is the goals that the user has told you till now

        {query}

        Your Answer:
        """
    goalsprompt = PromptTemplate(
    input_variables=["query"],
    template=purpose
    )
    goal_prompt = SystemMessagePromptTemplate(prompt=goalsprompt)
    chat_prompt = ChatPromptTemplate.from_messages([goal_prompt])
    goalsx = goalsdb.get()['documents'][:]
    query = ' '.join(goalsx)  
    result = llm(chat_prompt.format_prompt(query = query).to_messages())
    return result.content

### INTERVENTION BOT

In [147]:
def intervention_bot(query,trackingdb,userid):
    vectorstore_folder = f'db/{userid}'
    
    llm = ChatOpenAI(temperature=0.6)
    assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
    interventiondb = Chroma(collection_name=f'x{userid}intervention',persist_directory=f'{vectorstore_folder}/intervention', embedding_function=embeddings)
    
    purpose = """
        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
        You are an expert in Intervention techniques. Before starting the process always say this if yhe relevent history is empty:'Great, let’s begin the intervention. Please remember that this is a learning process and you may not get it right the first time. That’s okay, I’m here to help you and give you feedback and suggestions.'
        If the user for some reason says no to the intervention stage you can say:'That’s okay, I understand that you may not feel ready to start the intervention right now. You can come back to it later when you feel more comfortable. Just type “resume” or “continue” when you want to start the intervention.'
        
        FOR THE INTERVENTION STAGE THIS IS YOUR WORKFLOW

        Step 1: Identify the situation or trigger that causes you anxiety
        Step 2: Identify your thoughts or beliefs about the situation or trigger
        Step 3: Identify your feelings or emotions that result from your thoughts or beliefs
        Step 4: Identify your behaviors or actions that result from your feelings or emotions
        Step 5: Evaluate your thoughts or beliefs for evidence, accuracy, and helpfulness
        Step 6: Challenge your thoughts or beliefs by asking yourself some questions or generating some alternatives
        Step 7: Replace your thoughts or beliefs with more realistic and helpful ones
        Step 8: Identify the new feelings or emotions that result from your new thoughts or beliefs
        Step 9: Identify the new behaviors or actions that result from your new feelings or emotions
        Step 10: Review the outcome and feedback of your cognitive restructuring


        (This is an example of one of the steps)
        EXAMPLE: 
        Diya: let’s learn the first step of cognitive restructuring. The first step is to identify the situation or trigger that causes you anxiety.
        user: ok
        Diya: [explains everything related to it with examples] A situation or trigger is anything that makes you feel anxious or nervous in a social context. It can be a person, a place, an event, a task, a conversation, etc. For example, some common situations or triggers for social anxiety are:Meeting new peopleGiving a presentationGoing to a partyMaking a phone callAsking someone out
        user: ok i understand
        Diya: a possible situation or trigger for social anxiety is:Situation: I had to give a presentation at work in front of my boss and colleagues.Anxiety level before: 9/10Anxiety level after: 8/10
        user: ok
        Diya: Now it’s your turn. Think of a recent situation or trigger that caused you anxiety and write it down in the first column of your thought record. Don’t forget to rate your anxiety level before and after the situation.
        user: [user works with your example]
        Diya: Thank you for your answer. You have identified a situation or trigger that caused you anxiety and rated your anxiety level before and after the situation
        
        
        
        THIS IS THE CONVERSATION THAT SHOWS THE PROBLEMS THE USER IS FACING:
        {problems}
        
        
        USE RELEVENT PEICES OF HISTORY TO KEEP TRACK OF THE STEPS YOU NEED THERE ARE 10 STEPS SO KEEP TRACK OF THEM:
        {history}
        
        (IF YOU THINK THE USER IS READY TO MOVE TO THE NEXT STAGE OF GIVING HOMEWORK PROMPT THE USER FOR IT)
        
        IF YOU THINK THE USER IS READY TO MOVE TO THE NEXT STAGE THEN MOVE TO THE REVIEW STAGE TELL THE USER SHOULD I INITIATE THE REVIEW PHASE.
        This is the current conversation:
        user: {query}
        Diya:
        
        """
    interventionprompt = PromptTemplate(
    input_variables=["query","history","problems"],
    template=purpose
    )
    intervention_prompt = SystemMessagePromptTemplate(prompt=interventionprompt)
    chat_prompt = ChatPromptTemplate.from_messages([intervention_prompt])
    problems_list = assessmentdb.get()['documents'][:]
    history_list = interventiondb.get()['documents'][-10:]
    
    problems = ' '.join(problems_list) 
    history = ' '.join(history_list)  
    Chroma.add_texts(trackingdb,texts=["ASSESSMENT"])
    result = llm(chat_prompt.format_prompt(query=query,history=history,problems=problems).to_messages())
    Chroma.add_texts(interventiondb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nUser:{query}\nDiya:{result.content}"])
    return result.content

### FALLBACK BOT

In [148]:
def fallback_bot(query,result,trackingdb,userid,formatted_timestamp):
    
    vectorstore_folder = f'db/{userid}'
    Chroma.add_texts(trackingdb,texts=["INTERVENTION"])

    llm = ChatOpenAI(temperature=0.6)
    assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
    interventiondb = Chroma(collection_name=f'x{userid}intervention',persist_directory=f'{vectorstore_folder}/intervention', embedding_function=embeddings)
    vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
    goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)
    
    problems = assessmentdb.get()['documents'][:]
    intervention = interventiondb.get()['documents'][-10:]
    conversation = vectordb.get()['documents'][-10:]
    goals = goalsdb.get()['documents'][:]
    similar = vectordb.similarity_search(query)
    
        
    problems = ' '.join(problems) 
    intervention = ' '.join(intervention)  
    conversation=' '.join(conversation) 
    goals=' '.join(goals) 
    purpose = """
        This is the CURRENT DATE AND TIME: {time}
        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
        
        your purpose is to handle the user query with an appropriate answer. You are also an expert in providing review and conclusions
        Diya reviews the main points and takeaways from the session with the user and summarizes what they have learned and achieved. Diya also asks the user to rate their distress level again on a scale of 0 to 10 and compare it with their initial rating. Diya praises the user for their efforts and accomplishments and expresses confidence in their ability to cope with their problem. Diya also solicits feedback from the user about the session, such as what they liked, disliked, or found helpful or unhelpful.
        
        After completing a certain number of sessions or achieving their goals, help the patient to review their overall progress and achievements
        Help the patient to identify what they learned from the chatbot and how they can apply it in their future situations
        Help the patient to identify any potential challenges or setbacks that they may face and how they can cope with them
        Help the patient to develop a relapse prevention and maintenance plan that includes some strategies or reminders to keep up their new skills and habits
        Congratulate the patient for their efforts and accomplishments and thank them for using the chatbot
        Ask the patient for their feedback or suggestions on how to improve the chatbot
        Provide some resources or referrals for the patient if they need further support or assistance

        This is the previous conversation you have had with the user:
        {conversation}
        
        This is the problems that you have identified of the user:
        {problems}
        
        This contains intervation and homework conversation:
        {intervention}
        
        This is the goals conversation you have had with the user:
        {goals}
        
        
        This is the current convesation with the user
        user: {query}
        Diya:
        
        """
    fallback = PromptTemplate(
    input_variables=['time','conversation','problems','intervention','goals','query'],
    template=purpose
    )
    fallback_prompt = SystemMessagePromptTemplate(prompt=fallback)
    chat_prompt = ChatPromptTemplate.from_messages([fallback_prompt])

    result = llm(chat_prompt.format_prompt(time=formatted_timestamp,conversation=conversation, problems= problems, intervention=intervention,goals=goals,query=query).to_messages())
    Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nUser:{query}\nDiya:{result.content}"])
    return result.content

In [149]:
def chat(query,userid):
    create_progress_folder(userid)
    current_timestamp = time.time()
    formatted_timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_timestamp))
    userid = fix_collection_name(userid.lower())
    vectorstore_folder = f'db/{userid}'
    filename = f'progress/{userid}/progress.txt'

    if folder_exists(vectorstore_folder):
        vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
        trackingdb = Chroma(collection_name=f'x{userid}tracking',persist_directory=f'{vectorstore_folder}/tracking', embedding_function=embeddings)
        
        #***************************PHASE IDENTIFICATION***************************

        result = identification_bot(query,userid)

        
        #***************************INTRDOCUTON PHASE***************************
        
        if check_similarity(result,'INTRODUCTION'):
            response = introduction_bot(query,vectordb,formatted_timestamp,userid)
            return response,result
            
        #***************************ASSESSMENT PHASE***************************
    
        elif check_similarity(result,'ASSESSMENT'):
            check = False
            assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
            progress_file = f'progress/{userid}/progress.txt'
            questions = read_file(progress_file)
            if len(questions) == 0:
                response,check = Assessment_bot(query,vectordb,formatted_timestamp,userid,trackingdb)
                if not check:
                    Chroma.add_texts(assessmentdb,texts=[f"Human: {query}\n\n"])
                    assessmentdb.persist()
                    assessmentdb = None 
                assessment_docs = assessmentdb.get()['documents'][:]
                for i in assessment_docs:
                    Chroma.add_texts(vectordb,texts=[i])

                return response,result
            else:
                question = questions[0]
                remove_first_line(progress_file)
                Chroma.add_texts(assessmentdb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nDiya:{question}\n"])
                return question,result
                
            
            
        #***************************GOALS SETTING PHASE***************************
        
        elif check_similarity(result,'Goal setting'):
            response = goal_setter(query,userid)
            Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{result}\n\n"])
            return response,result

            
        
         #***************************GOALS RETRIVER PHASE***************************
        
        elif check_similarity(result,'Goal retriver'):
            response = goals_retriver(userid)
            Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman:{query}\nDiya:{result}\n"])
            return response,result

            
         #***************************INTERVENTION PHASE***************************
            
        elif check_similarity(result,'Intervention'):
            response = intervention_bot(query,trackingdb,userid)
            return response,result
            
            
        #***************************FALLBACK PHASE***************************
            
        else:
#             response = fallback_bot(query,result,trackingdb,userid,formatted_timestamp)
            Chroma.add_texts(vectordb,texts=[f"\n===============================\nConversation-Time-Stamp:{formatted_timestamp}\nHuman: {query}\nDiya:{result}\n\n"])
            
            return result,'FRONTEND'
            
            
    else:
        #Here is Introductory phase
        create_progress_folder(userid)
        vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
        response = introduction_bot(query,vectordb,formatted_timestamp,userid)
        vectordb.persist()
        vectordb = None
        with open("indexholders.txt", "a", encoding="utf-8") as f:
            f.write(userid + "\n")
        return response,'INTRODUCTION'



In [176]:
while True:
    query = input("Ask: ")
    if query.lower()=='quit':
        break
    response,stage = chat(query,'15sBCSDadsosDS')
    print(f'{response}\n{stage}\n\n')

Ask: yea so for 10 days I have been depressed


Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds
Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds/tracking
Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds/assessment
Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds/intervention
Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds
Using embedded DuckDB with persistence: data will be stored in: db/15sbcsdadsosds/goalsdb


Based on what you've told me, it sounds like you might benefit from the assessment phase to identify the root of the problem. Would you like to proceed with that?
FRONTEND


Ask: quit


### FRONTEND BOT

In [100]:
def frontend_bot(query,userid):
    current_timestamp = time.time()
    formatted_timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_timestamp))
            
    problems = ' '
    intervention = ' ' 
    conversation=' '
    goals=' ' 
    similar = ' '
    
    llm = ChatOpenAI(temperature=0)
    
    users = read_file('indexholders.txt')
    if userid in users:
        assessmentdb = Chroma(collection_name=f'x{userid}assessment',persist_directory=f'{vectorstore_folder}/assessment', embedding_function=embeddings)
        interventiondb = Chroma(collection_name=f'x{userid}intervention',persist_directory=f'{vectorstore_folder}/intervention', embedding_function=embeddings)
        vectordb = Chroma(collection_name=f'x{userid}',persist_directory=vectorstore_folder, embedding_function=embeddings)
        goalsdb = Chroma(collection_name=f'x{userid}goalsdb',persist_directory=f'{vectorstore_folder}/goalsdb', embedding_function=embeddings)

        problems = ' '.join([doc.page_content for doc in assessmentdb.similarity_search(query,k=7)])
        intervention = ' '.join([doc.page_content for doc in interventiondb.similarity_search(query,k=7)])
        goals = ' '.join([doc.page_content for doc in goalsdb.similarity_search(query,k=7)])
        similar = ' '.join([doc.page_content for doc in vectordb.similarity_search(query,k=7)])


    purpose = """
        This is the CURRENT DATE AND TIME: {time}
        Have a conversation with a human, answering the following questions as best you can.
        Your name is Diya and you are a wellness coach. You are 25 year old female and you are kind and open hearted and chats in friendly tone that allows users to confide in you. You are an expert in Cognitive Behavioral Therapy (CBT) principles and understand all the stages involved in CBT to help the uses.
        
        your purpose is to handle the user query with an appropriate answer. You are also an expert in providing review and conclusions
        
        CONVERSATION STAGE/INTRODUCTION STAGE
        This is the previous conversation you have had with the user:
        {similar}
        
        ASSESMENT STAGE
        This is the problems that you have identified of the user:
        {problems}
        
        INTERVENTION STAGE
        This contains intervation and homework conversation:
        {intervention}
        
        GOAL SETTING STAGE
        This is the goals conversation you have had with the user:
        {goals}

        
        This is the current convesation with the user
        user: {query}
        Diya:
        (IF YOU THINK INFORMATION IS LESS OR A STAGE HAS LESS INFO RETURN: 'NA')
        """
    fallback = PromptTemplate(
    input_variables=['time','problems','intervention','goals','query','similar'],
    template=purpose
    )
    fallback_prompt = SystemMessagePromptTemplate(prompt=fallback)
    chat_prompt = ChatPromptTemplate.from_messages([fallback_prompt])

    response = llm(chat_prompt.format_prompt(time=formatted_timestamp, problems= problems, intervention=intervention,goals=goals,query=query,similar=similar).to_messages())
    if check_similarity(response.content,'NA'):
        response,stage = chat(query,userid,formatted_timestamp)
        return response,stage
    else:
        return response.content,'DISCUSSION'